In [ ]:
# 几乎全部使用3*3 的卷积核以及2*2 的池化层，使用小的卷积核 进行多层堆叠和一个大的卷积核的感受野是相同的。小的卷积核还可以减少参数


In [1]:
import torch
from torch import nn
import numpy as np
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [10]:
# 定义Vgg 的block ，传入三个参数：依次是模型的层数，输入的通道数，输出的通道数，
def vgg_block(num_convs,in_channels,out_channels):
    net=[nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1),nn.ReLU(True)] # 定义第一层
    for i in range(num_convs-1):# 定义后面的很多层
        net.append(nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1))
        net.append(nn.ReLU(True))
    net.append(nn.MaxPool2d(2,2)) # 定义池化层
    return nn.Sequential(*net)
        

In [11]:
# 打印结构：
block_demo=vgg_block(3,64,128)
print(block_demo)

Sequential(
  (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [12]:
# 定义输入为（1，64，300，300）
input_demo=Variable(torch.zeros(1,64,300,300))
output_demo=block_demo(input_demo)
print(output_demo.shape)

torch.Size([1, 128, 150, 150])


In [15]:
# 经过一个Vgg block 输入大小被减半，通道数变成128
# 定义一个函数对这个vgg block 进行堆叠
def vgg_stack(num_convs,channels):
    net=[]
    for n,c in zip(num_convs,channels):
        in_c=c[0]
        out_c=c[1]
        net.append(vgg_block(n,in_c,out_c))
    return nn.Sequential(*net)

In [16]:
# 作为实例，定义一个稍微简单一点的vgg 结构，其中有8个卷积层
vgg_net=vgg_stack((1,1,2,2,2),((3,64),(64,128),(128,256),(256,512),(512,512)))
print(vgg_net)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=

In [17]:
# 有5个最大池化，说明会让图片的大小减少5倍，
test_x=Variable(torch.zeros(1,3,256,256))
test_y=vgg_net(test_x)
print(test_y.shape)

torch.Size([1, 512, 8, 8])


In [19]:
# 加上几层全连接层
class vgg(nn.Module):
    def __init__(self):
        super(vgg,self).__init__()
        self.feature=vgg_net
        self.fc=nn.Sequential(
            nn.Linear(512,100),
            nn.ReLU(True),
            nn.Linear(100,10)
        
        
        )
    def forward(self,x):
        x=self.feature(x)
        x=x.view(x.shape[0],-1)
        x=self.fc(x)
        return x